# Test

# 一開始先準備好原始 dataset 這是跨 project 的 common 部分

In [1]:
%run ../../../forth.py

reDef unknown
reDef -->
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.



In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
raw_data = pd.read_excel('Copy of Result_Rosa_NB -20210609 Tool.xlsx', sheet_name=0)  # 沒有指定就是 worksheet #0 
raw_data

C:\Users\8304018\AppData\Local\Continuum\anaconda3\envs\peforthDevelop\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  """Entry point for launching an IPython kernel.


,PSR Number,Title,Status,P1(What_Happen),P2(What_Device),tool_P3(What_Environment),P4(Function)
0,BITS454665,mapping Rosa BC project SYS_CHG_MIN_INPUT_CU...,Complete,NaN,NaN,NaN,NaN
1,BITS454673,Wistron - Update HellCat TGL- Intel RST PreOS ...,Complete,NaN,NaN,NaN,NaN
2,BITS454678,[CSMB CY20] Update SET_SYS_DPM_DERATING condition,Complete,NaN,NaN,NaN,NaN
3,BITS454679,CBA-2638: PCR6610/PCR6641 : Enable a BIOSConne...,Complete,NaN,NaN,bios,bios
4,BITS454680,"Cypress CCG5 PD FW Release Update, 01.04.64.30",Complete,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1412,NaN,HCT_3/5U_ENG: SUT show black screen after unpl...,Analyze,"halt, shows in",vga,NaN,NaN
1413,NaN,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",WAIT,shows in,dgfx,tool,NaN
1414,NaN,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",WAIT,shows in,NaN,tool,NaN
1415,NaN,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",WAIT,shows in,dgfx,tool,NaN


In [4]:
# James 有整理好各種 key names 放在另一張 work sheet #1
raw_targets = pd.read_excel('Copy of Result_Rosa_NB -20210609 Tool.xlsx',sheet_name=1)
raw_targets

C:\Users\8304018\AppData\Local\Continuum\anaconda3\envs\peforthDevelop\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  


,P1,P1_Count,P2,P2_Count,P3,P3_Count,P4,P4_Count
0,shows in,368,NaN,774.0,NaN,623.0,NaN,454.0
1,NaN,198,storage,103.0,bios,221.0,s3/s4/s5,200.0
2,halt,124,usb device,78.0,os,193.0,long run,156.0
3,error message,110,type-c,75.0,application,144.0,perfacemace,98.0
4,not match the specification,93,audio,57.0,tool,106.0,bios menu,63.0
...,...,...,...,...,...,...,...,...
184,show 0x806c0,1,NaN,NaN,NaN,NaN,NaN,NaN
185,is same as 50%,1,NaN,NaN,NaN,NaN,NaN,NaN
186,portrait display,1,NaN,NaN,NaN,NaN,NaN,NaN
187,couldn't preview,1,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# P2 devices 的總集合
devices = raw_targets[['P2','P2_Count']].set_index('P2').to_dict()['P2_Count']
del(devices[np.nan]) # 其中有 NaN 項如上列，要注意。
%f devices count nip --> 1 ( count ) # devices 含 device name 與 sampled count 將來小於 20 的必須捨棄
for k,v in devices.items():
    %f dup . space 1+ k . space v . cr


devices count nip --> 37 (<class 'int'>)
devices 含 device name 與 sampled count 將來小於 20 的必須捨棄

1 storage 103.0
2 usb device 78.0
3 type-c 75.0
4 audio 57.0
5 mouse/keyboard 55.0
6 vga 52.0
7 tbt 39.0
8 lcd 35.0
9 battery 26.0
10 cpu 25.0
11 dimm 24.0
12 hdmi 19.0
13 nic/network 19.0
14 camera 18.0
15 lid 16.0
16 type-c device 16.0
17 dgfx 16.0
18 dock 15.0
19 bluetooth 14.0
20 tpm 14.0
21 dp 13.0
22 touch lcd 12.0
23 ac adapter 9.0
24 pen 8.0
25 odd 7.0
26 monitor 6.0
27 wlan 6.0
28 tbt device 4.0
29 peg 3.0
30 mediacard/sd card 3.0
31 spi 2.0
32 ec 2.0
33 fingerprint 2.0
34 sensors 2.0
35 sata 2.0
36 bluetooth device 1.0
37 uhd lcd 1.0


In [7]:
# P2 devices 實例大於 20 者
devices = raw_targets[['P2','P2_Count']].set_index('P2').to_dict()['P2_Count']
del(devices[np.nan]) # 其中有 NaN 項如上列，要注意。
%f devices count nip --> 1 ( count ) # devices 含 device name 與 sampled count 將來小於 20 的必須捨棄
for k,v in devices.items():
    if v >= 20 : 
        %f dup . space 1+ k . space v . cr


devices count nip --> 37 (<class 'int'>)
devices 含 device name 與 sampled count 將來小於 20 的必須捨棄

1 storage 103.0
2 usb device 78.0
3 type-c 75.0
4 audio 57.0
5 mouse/keyboard 55.0
6 vga 52.0
7 tbt 39.0
8 lcd 35.0
9 battery 26.0
10 cpu 25.0
11 dimm 24.0


In [8]:
# 拆分 P2 成個別 device 的 list 準備作為 target column 
classes = []
for dd in raw_data['P2(What_Device)']:
    row = []
    for i in range(len(devices)):
        if type(dd) == str and list(devices.keys())[i] in dd:  # dd 有的是 np.nan 屬 float 要排除
            row.append(1)
        else:
            row.append(0)
    classes.append(row)
%f classes count nip -->
%f raw_data count nip -->
%f classes :> [7] -->

classes count nip --> 1417 (<class 'int'>)
raw_data count nip --> 1417 (<class 'int'>)
classes :> [7] --> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (<class 'list'>)


In [10]:
targets = pd.DataFrame(np.array(classes), columns = devices)  # 這裡挺妙，columns 直接餵 devices 即可，不必弄成 list(devices.keys()).
targets

,storage,usb device,type-c,audio,mouse/keyboard,vga,tbt,lcd,battery,cpu,...,tbt device,peg,mediacard/sd card,spi,ec,fingerprint,sensors,sata,bluetooth device,uhd lcd
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1413,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
data = pd.merge(raw_data['Title'],targets, left_index=True, right_index=True)
data 

,Title,storage,usb device,type-c,audio,mouse/keyboard,vga,tbt,lcd,battery,...,tbt device,peg,mediacard/sd card,spi,ec,fingerprint,sensors,sata,bluetooth device,uhd lcd
0,mapping Rosa BC project SYS_CHG_MIN_INPUT_CU...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wistron - Update HellCat TGL- Intel RST PreOS ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,[CSMB CY20] Update SET_SYS_DPM_DERATING condition,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CBA-2638: PCR6610/PCR6641 : Enable a BIOSConne...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Cypress CCG5 PD FW Release Update, 01.04.64.30",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412,HCT_3/5U_ENG: SUT show black screen after unpl...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1413,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1414,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1415,"HCT_5U_ENG_MDAVT: DG1_SUT show ""No base driver...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
import datarobot as dr

In [14]:
dr.Client(token=peforth.execute('api_token').pop(), endpoint=peforth.execute('endpoint').pop())

# 對某 device 發動專屬 project 做 prediction 並檢查對錯

符合 >20 samples 的 devices 有這些 

     1 storage       103.0
     2 usb device     78.0
     3 type-c         75.0
     4 audio          57.0
     5 mouse/keyboard 55.0
     6 vga            52.0
     7 tbt            39.0
     8 lcd            35.0
     9 battery        26.0
    10 cpu            25.0
    11 dimm           24.0


In [85]:
# 針對每一種 device 開特定 project 去分辨
project_name = 'dimm' 

In [86]:
project_id = dr.Project.list(search_params={'project_name':project_name})[0].id
project = dr.Project.get(project_id=project_id)
%f project_id -->
%f project -->


project_id --> 60d1ed7f3d8429e3f2cdb6e8 (<class 'str'>)
project --> Project(QT TR Title Parsing-dimm) (<class 'datarobot.models.project.Project'>)


- 如上述：不行，recommended model 可能都沒有 holdout 可用了。直接用 project.get_models() 好了。

In [87]:
models = project.get_models()
%f models :> [0] --> # The first one is the recommended model but probably is_frozen
%f models :> [0].is_frozen -->
%f models :> [1].is_frozen -->
%f models :> [2].is_frozen -->


models :> [0] --> Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)') (<class 'datarobot.models.model.Model'>)
The first one is the recommended model but probably is_frozen

models :> [0].is_frozen --> True (<class 'bool'>)
models :> [1].is_frozen --> False (<class 'bool'>)
models :> [2].is_frozen --> False (<class 'bool'>)


In [88]:
# Calculate new training predictions on holdout partition of dataset
training_predictions_job = models[1].request_training_predictions(dr.enums.DATA_SUBSET.HOLDOUT)
training_predictions = training_predictions_job.get_result_when_complete()
training_predictions


In [89]:
# Fetch training predictions as data frame
df_predict_holdout = training_predictions.get_all_as_dataframe()
df_predict_holdout

,row_id,partition_id,prediction,class_1.0,class_0.0
0,0,Holdout,0.0,0.000112,0.999888
1,4,Holdout,0.0,0.000153,0.999847
2,22,Holdout,0.0,0.000295,0.999705
3,29,Holdout,0.0,0.002666,0.997334
4,35,Holdout,0.0,0.001251,0.998749
...,...,...,...,...,...
278,1379,Holdout,0.0,0.004084,0.995916
279,1386,Holdout,0.0,0.006913,0.993087
280,1391,Holdout,0.0,0.001526,0.998474
281,1403,Holdout,0.0,0.000295,0.999705


In [90]:
# 瞧一瞧所有的 prediction 結果。等下要拿來跟 dataset 比對。
%f df_predict_holdout :> ['prediction'] list --> cr
%f df_predict_holdout :> ['row_id'] list -->
    

df_predict_holdout :> ['prediction'] list --> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [91]:
# prediction 結果跟 dataset 比對。用 prediction 0 的 index 去算 dataset 的 sum 就是 FN
indexs = list(df_predict_holdout.query('prediction == 0')['row_id'])
%f indexs count nip -->
positives = [data[project_name].iloc[i] for i in indexs]    
%f positives -->
sum(positives)


indexs count nip --> 279 (<class 'int'>)
positives --> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (<class 'list'>)


0

In [92]:
# prediction 結果跟 dataset 比對。用 prediction 1 的 index 去算 dataset 的 sum 就是 TP
indexs = list(df_predict_holdout.query('prediction == 1')['row_id'])
%f indexs count nip -->
positives = [data[project_name].iloc[i] for i in indexs]    
%f positives -->
sum(positives)


indexs count nip --> 4 (<class 'int'>)
positives --> [1, 1, 1, 1] (<class 'list'>)


4